In [1]:
import pandas as pd
import numpy as np
from numpy import random
from random import randrange
from matplotlib import pyplot
from statsmodels.tsa.seasonal import seasonal_decompose
from matplotlib import pyplot
from pandas import Series
from random import seed
from random import random
from statsmodels.tsa.stattools import adfuller



import matplotlib.pyplot as plt

C:\Users\murat.ozemre\AppData\Local\Continuum\Anaconda3\envs\Tensor\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [13]:
from __future__ import print_function
import os
import sys
#os.chdir('C:/Users/murat.ozemre/Documents/MOZEMRE-OZEL/Doktora/2017 Tez/Veri Analizi/Ver 3 Tez Izleme Calısmaları/Auto Correlation')
#os.chdir('C:/Users/murat.ozemre/Documents/MOZEMRE-OZEL/Doktora/2017 Tez/Veri Analizi/Ver 3 Tez Izleme Calısmaları')

os.chdir('C:/Users/murat.ozemre/Desktop/Thesis_Project/Data_Sources_and_Preparation')

In [14]:
Scaler_Type_Options = ['Normalizer', 'MinMaxScaler','MaxAbsScaler','RobustScaler','StandardScaler' ]
#Scaler_Type_Options = ['Normalizer', 'MinMaxScaler' ]

Product_Type_Options = [841810,841840,841850]
Product=Product_Type_Options[2]
ScalerType=Scaler_Type_Options[2]
print(Product,ScalerType)

MonthSeries="1"
MonthSeries_option=["1","12"]
#MonthSeries_option=["12","123","1236","1236_12","__12","__126","__1263","__12632"]
#MonthSeries_option=["12","123","1236","1236_12","__12"]


841850 MaxAbsScaler


In [4]:
def setpartner(Product):
    if Product==841810 :
        IMP1="USA"
        IMP2="JPN"
        IMP3="FRA"
        IMP4="AUS"
        EXP1="POL"
        EXP2="KOR"
        EXP3="ITA"
    elif Product==841840 :
        IMP1="USA"
        IMP2="JPN"
        IMP3="FRA"
        IMP4="DEU"
        EXP1="DEU"
        EXP2="NLD"
        EXP3="HUN"
    elif Product==841850 :
        IMP1="USA"
        IMP2="AUS"
        IMP3="ITA"
        IMP4="IDN"
        EXP1="AUT"
        EXP2="CHZ"
        EXP3="ITA"
    
    return IMP1,IMP2,IMP3,IMP4,EXP1,EXP2,EXP3

In [8]:
def readexcel(Product):
    if Product==841810 :
        Data_Core = pd.ExcelFile('Out_841810_CHN_UK_Data_Core_1236_12.xlsx')
        Data_Core = Data_Core.parse('Sheet1', header=0,index_col=None, na_values=['NA'])
    elif Product==841840:
        Data_Core = pd.ExcelFile('Out_841840_CHN_UK_Data_Core_1236_12.xlsx')
        Data_Core = Data_Core.parse('Sheet1', header=0,index_col=None, na_values=['NA'])
    elif Product==841850:
        Data_Core = pd.ExcelFile('Out_841850_CHN_UK_Data_Core_1236_12.xlsx')
        Data_Core = Data_Core.parse('Sheet1', header=0,index_col=None, na_values=['NA'])
    return Data_Core

In [9]:
def readexcel_hat(Product):
    if Product==841810 :
        Data_Core = pd.ExcelFile('Out_841810_CHN_UK_Data_Core_1236_12.xlsx')
        Data_Core = Data_Core.parse('Sheet2', header=0,index_col=None, na_values=['NA'])
    elif Product==841840:
        Data_Core = pd.ExcelFile('Out_841840_CHN_UK_Data_Core_1236_12.xlsx')
        Data_Core = Data_Core.parse('Sheet2', header=0,index_col=None, na_values=['NA'])
    elif Product==841850:
        Data_Core = pd.ExcelFile('Out_841850_CHN_UK_Data_Core_1236_12.xlsx')
        Data_Core = Data_Core.parse('Sheet2', header=0,index_col=None, na_values=['NA'])
    Data_Core=Data_Core.iloc[3:,:]
    return Data_Core

In [10]:
def set_y(Product,Data_Core):
    if Product==841810 :
        y=Data_Core['CHN_GBR_841810']        
    elif Product==841840 :
        y=Data_Core['CHN_GBR_841840']
    elif Product==841850 :
        y=Data_Core['CHN_GBR_841850']
    return y

In [11]:
def set_X(Product,Data_Core,MonthSeries):
    if MonthSeries=="1":
        X=Data_Core[
    
            ['CHN_World_{}-1'.format(Product), 'CHN_{}_{}-1'.format(IMP1,Product), 'CHN_{}_{}-1'.format(IMP2,Product),
               'CHN_{}_{}-1'.format(IMP3,Product), 'CHN_{}_{}-1'.format(IMP4,Product), 'CHN_GBR_{}-1'.format(Product),
               '{}_GBR_{}-1'. format(EXP1,Product), '{}_GBR_{}-1'. format(EXP2,Product), '{}_GBR_{}-1'. format(EXP3,Product), 
               'EPU_CHN-1','EPU_UK-1', 'EPU_World-1', 
               'BCI_CHN-1', 'CCI_CHN-1', 'CLI_CHN-1','GDP_CHN-1', 
               'BCI_GBR-1', 'CCI_GBR-1', 'CLI_GBR-1', 'GDP_GBR-1',
               'PPI_CHN-1', 'PPI_GBR-1', 'CPI_CHN-1', 'CPI_GBR-1', 
               'CNY-1', 'GBP-1','World-1','Date']
        ]
        
        
    return X

In [ ]:


# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return Series(diff)

In [9]:

#y=set_y(Product,Data_Core)
#X=set_X(Product,Data_Core,MonthSeries)

for Product in Product_Type_Options:
    partner=setpartner(Product)
  
    IMP1=partner[0]
    IMP2=partner[1]
    IMP3=partner[2]
    IMP4=partner[3]
    EXP1=partner[4]
    EXP2=partner[5]
    EXP3=partner[6]
    
    Data_Core=readexcel(Product)
    Data_Core_hat=readexcel_hat(Product)
    print(Product)
    if Product==841810 :
        y1=set_y(Product,Data_Core)
        X1=set_X(Product,Data_Core,MonthSeries)
        
        y1hat=set_y(Product,Data_Core_hat)
        X1hat=set_X(Product,Data_Core_hat,MonthSeries)
        
    elif Product==841840 :
        y2=set_y(Product,Data_Core)
        X2=set_X(Product,Data_Core,MonthSeries)
        y2hat=set_y(Product,Data_Core_hat)
        X2hat=set_X(Product,Data_Core_hat,MonthSeries)
        
    elif Product==841850 :
        y3=set_y(Product,Data_Core)
        X3=set_X(Product,Data_Core,MonthSeries)

        y3hat=set_y(Product,Data_Core_hat)
        X3hat=set_X(Product,Data_Core_hat,MonthSeries)

841810
841840
841850


In [ ]:
#Mean Staitionary testi için 2 ayrı setin Mean Farkı 
Z=y1
split = int(len(Z)/ 2)
Z1, Z2 = Z[0:split], Z[split:]
mean1, mean2 = Z1.mean(), Z2.mean()
var1, var2 = Z1.var(), Z2.var()
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))

In [ ]:
# calculate and plot a white noise series
from random import gauss
from random import seed
from pandas import Series
from pandas.plotting import autocorrelation_plot
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.gofplots import qqplot


from statsmodels.tsa.stattools import adfuller
from random import randrange

# seed random number generator
seed(1)
# create white noise series
series = [gauss(0.0, 1.0) for i in range(1000)]
series = Series(series)
# summary stats
print(series.describe())
# line plot
series.plot()
#pyplot.savefig("Deneme.png", format='png', dpi=300)

pyplot.show()

# histogram plot
series.hist()
pyplot.show()
# autocorrelation
autocorrelation_plot(series)
pyplot.show()

In [ ]:
def Analiz(z):
    a=''
    pyplot.gcf().clear()
    print(z.describe())
    z.plot()
    pyplot.title('{}{} Line Plot'.format(z.name, a))
    pyplot.savefig("{}{} Line Plot.png".format(z.name, a), format='png', dpi=300)
    pyplot.show()
    
    z.hist()
    pyplot.title('{}{} Histogram Plot'.format(z.name, a))
    pyplot.savefig("{}{} Histogram Plot.png".format(z.name, a), format='png', dpi=300)
    pyplot.show()

    autocorrelation_plot(z)
    pyplot.title('{}{} Auto Correlation Plot'.format(z.name, a))
    pyplot.savefig("{}{} Auto Correlation Plot.png".format(z.name, a), format='png', dpi=300)
    pyplot.show()
    
    plot_acf(z,lags=30)
    pyplot.title('{}{} Auto Correlation with legs'.format(z.name, a))
    pyplot.savefig("{}{} Auto Correlation with legs .png".format(z.name, a), format='png', dpi=300)
    pyplot.show()
    
    plot_pacf(z,lags=30)
    pyplot.title('{}{} Partial Auto Correlation with legs'.format(z.name, a))
    pyplot.savefig("{}{} Partial Auto Correlation with legs .png".format(z.name, a), format='png', dpi=300)
    pyplot.show()
    
    return 

In [ ]:
def Stationary_Analiz(z):
    a=''
    print(z.name)
    result = adfuller(z)
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

    series = [i+randrange(10) for i in range(1,100)]
    modeltype='additive'
    
    result = seasonal_decompose(z.tolist(), model=modeltype, freq=12)
    result.plot()
   
    pyplot.savefig("{}{} Decompose with {} moded .png".format(z.name,a,modeltype), format='png', dpi=300)

    pyplot.show()
    return

In [ ]:
def Stationary_Analiz_Short(z):
    #print(Product)
    result = adfuller(z)
    adf=int(100000*result[0])/100000
    p_value=int(100000*result[1])/100000

    #print('ADF Statistic: %f' % adf)
    #print('p-value: %f' % p_value)
    
    return adf,p_value

In [ ]:
z=y1.dropna()
series = [i+randrange(10) for i in range(1,100)]
result = seasonal_decompose(z.tolist(), model='multiplicative', freq=12)
result.plot()
pyplot.show()

observed=pd.Series(data=result.observed, index=z.index)
trend=pd.Series(data=result.trend, index=z.index)
seasonal=pd.Series(data=result.seasonal, index=z.index)
resid=pd.Series(data=result.resid, index=z.index)

In [ ]:
resid=resid.dropna()
resid.plot()
pyplot.show()
Stationary_Analiz_Short(resid)

In [ ]:
z=y1.dropna()
series = [i+randrange(10) for i in range(1,100)]
result = seasonal_decompose(z.tolist(), model='additive', freq=12)
result.plot()
pyplot.show()

observed=pd.Series(data=result.observed, index=z.index)
trend=pd.Series(data=result.trend, index=z.index)
seasonal=pd.Series(data=result.seasonal, index=z.index)
resid=pd.Series(data=result.resid, index=z.index)

In [ ]:
resid=resid.dropna()
resid.plot()
pyplot.show()
Stationary_Analiz_Short(resid)

In [ ]:
print(observed[10],trend[10],seasonal[10],resid[10])

In [ ]:
z=y2.dropna()
series = [i+randrange(10) for i in range(1,100)]
result = seasonal_decompose(z.tolist(), model='multiplicative', freq=12)
result.plot()
pyplot.show()

In [ ]:
z=y3.dropna()
series = [i+randrange(10) for i in range(1,100)]
result = seasonal_decompose(z.tolist(), model='multiplicative', freq=12)
result.plot()
pyplot.show()


In [ ]:
#ADF Testi herbir özellik için
#Hangi degerin teste girecğini belirlemek için

X=X1.dropna() 
Xhat=X1hat.dropna()

#   Herbir özellik için ADF Calısır

n_feature=X.shape[1]
sonuc=[]
Stationary_sonuc=np.zeros((4,X.shape[1]))

for i in range(n_feature):
        
        x=X.iloc[:, i]
        xhat=Xhat.iloc[:, i]
        title=list(X)[i:i+1]
       
        analiz_sonuc_x=Stationary_Analiz_Short(x)
        analiz_sonuc_xhat=Stationary_Analiz_Short(xhat)
        
        sonuc.append(title)
        sonuc.append(analiz_sonuc_x)
        sonuc.append(analiz_sonuc_xhat)

sonuc_array=np.asarray(sonuc).reshape(n_feature,3)

sonuc_array


In [ ]:
df = pd.DataFrame(np.random.randn(28,5),columns='Title A P Ahat Phat'.split())
#sonuc=np.asarray(sonuc).reshape(n_feature,3)  
for i in range(n_feature):   

#        print(str(sonuc[0+3*i]))
#        print (sonuc[1+3*i][0])
#        print (sonuc[1+3*i][1])
#        print (sonuc[2+3*i][0])
#        print (sonuc[2+3*i][1])
        df.iloc[i, 0]=str(sonuc[0+3*i])
        df.iloc[i,1]=sonuc[1+3*i][0]
        df.iloc[i, 2]=sonuc[1+3*i][1]
        df.iloc[i, 3]=sonuc[2+3*i][0]
        df.iloc[i, 4]=sonuc[2+3*i][1]
df.to_excel('Stationary_Analiz_X1.xlsx',index = False)

In [ ]:
#df = pd.DataFrame(randn(28,5),columns='Title A P Ahat Phat'.split())
df

In [ ]:
i=25
df.iloc[0, 4]=str(sonuc[0+3*i])
df.iloc[0, 4]
df.iloc[i-1, 1]=sonuc[1+3*i][0]
print(df.iloc[24, 1])
print(sonuc[76][0])

In [ ]:
# O sutunun adını donuyor.
print(list(X1)[15:16])
# O sutunun verisini donuyor
#X1.iloc[:, 15]

In [ ]:
list(X1)[15:16]

In [ ]:
Analiz(X1.iloc[:, 12])

In [ ]:
def ts_diagnostics(y, lags=None, title=''):
    '''
    Calculate acf, pacf, qq plot and Augmented Dickey Fuller test for a given time series
    '''
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
        
    # weekly moving averages (5 day window because of workdays)
    rolling_mean = pd.rolling_mean(y, window=24)
    rolling_std = pd.rolling_std(y, window=24)
    
    fig = plt.figure(figsize=(14, 12))
    layout = (3, 2)
    ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
    acf_ax = plt.subplot2grid(layout, (1, 0))
    pacf_ax = plt.subplot2grid(layout, (1, 1))
    qq_ax = plt.subplot2grid(layout, (2, 0))
    hist_ax = plt.subplot2grid(layout, (2, 1))
    
    # time series plot
    y.plot(ax=ts_ax)
    rolling_mean.plot(ax=ts_ax, color='crimson');
    rolling_std.plot(ax=ts_ax, color='darkslateblue');
    plt.legend(loc='best')
    ts_ax.set_title(title, fontsize=24);
    
    # acf and pacf
    plot_acf(y, lags=lags, ax=acf_ax, alpha=0.5)
    plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.5) 
    
    # qq plot
    qqplot(y, line='s', ax=qq_ax)
    qq_ax.set_title('QQ Plot')
    
    # hist plot
    y.plot(ax=hist_ax, kind='hist', bins=25);
    hist_ax.set_title('Histogram');
    plt.tight_layout();
    plt.savefig("{}Auto Correlation Study .png".format(z.name), format='png', dpi=300)
    plt.show()
    
    # perform Augmented Dickey Fuller test
    print('Results of Dickey-Fuller test:')
    dftest = adfuller(y, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['test statistic', 'p-value', '# of lags', '# of observations'])
    for key, value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)
    return



In [ ]:
z=y3
y_diff = np.diff(z)
#y_diff=z
ts_diagnostics(y_diff, lags=30, title='Auto Correlation Study for {}hat'.format(z.name))


In [ ]:
z=y3
result = adfuller(z,maxlag=13)
#result = adfuller(z,maxlag=13)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print(result[4])

#result = adfuller(z,maxlag=13,regression='ct',autolag='t-stat')
result = adfuller(z,maxlag=13,autolag='AIC')
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print(result[4])


result = adfuller(z)
#result = adfuller(z,maxlag=24,regression='ct',autolag='t-stat')
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print(result[4])

In [ ]:
z=y1
result = adfuller(z,maxlag=24,regression='ct')

#result = adfuller(z,maxlag=24,regression='ct',autolag='t-stat')
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print(result[4])


In [ ]:
z=y1

result = adfuller(z,maxlag=24,regression='ct',autolag='t-stat')
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print(result[4])

In [ ]:
a=y1.dropna()
#Analiz(X.iloc[:, 12])
Analiz(a)
Stationary_Analiz(a)



In [ ]:
a=y2.dropna()
#Analiz(X.iloc[:, 0])
Analiz(a)
Stationary_Analiz(a)


In [ ]:
a=y3
#Analiz(X.iloc[:, 0])
Analiz(a)
Stationary_Analiz(a)


In [ ]:
a=y1hat.dropna()
#Analiz(X.iloc[:, 0])
Analiz(a)
Stationary_Analiz(a)

In [ ]:
a=y2hat.dropna()
#Analiz(X.iloc[:, 0])
Analiz(a)
Stationary_Analiz(a)

In [ ]:
a=y3hat.dropna()
#Analiz(X.iloc[:, 0])
Analiz(a)
Stationary_Analiz(a)

In [ ]:
# calculate the stationarity of a random walk
from random import seed
from random import random
from statsmodels.tsa.stattools import adfuller
#result = adfuller(X.iloc[:, 20])
print(Product)
result = adfuller(y1)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))
# Section 12.4
# additive decompose a contrived additive time series
from random import randrange
from matplotlib import pyplot
from statsmodels.tsa.seasonal import seasonal_decompose
from matplotlib import pyplot

series = [i+randrange(10) for i in range(1,100)]
result = seasonal_decompose(y1.tolist(), model='additive', freq=12)

result.plot()

pyplot.show()
plt.title('Page One')


#result = seasonal_decompose(y.tolist(), model='multiplicative', freq=12)
#result.plot()
#pyplot.show()

In [ ]:
#result = seasonal_decompose(y.tolist(), model='additive', freq=12)
#print(result.trend)
#print(result.seasonal)
#print(result.resid)
#print(result.observed)


In [ ]:
# ACF plot of time series
from pandas import Series
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
#series = Series.from_csv('daily-minimum-temperatures.csv', header=0)
print(Product)
plot_acf(y1,lags=30)
pyplot.show()

In [ ]:
import seaborn as sns
%matplotlib inline

In [ ]:
a=pd.DataFrame(columns=['y1'],data=y1.values)
b=pd.DataFrame(columns=['y1hat'],data=y1hat.values)
#c=pd.concat([a, b], axis=1)

#sns.pairplot(c[['y1','y1hat']])

In [ ]:
print(y1[4],y1[3],y1hat[3],y1[4]-y1[3])

In [ ]:
y1hat


In [ ]:
int

In [ ]:
# ARIMA ICIN 

In [15]:
# grid search ARIMA parameters for a time series
import warnings
from pandas import Series
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt
# evaluate an ARIMA model for a given order (p,d,q) and return RMSE

def evaluate_arima_model(X, arima_order):
    # prepare training dataset
    X = X.astype('float32')
    train_size = int(len(X) * 0.50)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    # make predictions
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        # model_fit = model.fit(disp=0)
        model_fit = model.fit(trend='nc', disp=0)
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
        # calculate out of sample error
    rmse = sqrt(mean_squared_error(test, predictions))
    r2=r2_score(test, predictions)
    return rmse,test,predictions

# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score = float("inf")
    best_cfg = float("inf")
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    result = evaluate_arima_model(dataset, order)
                    rmse = result[0]
                    best_test=result[1]
                    best_prediction=result[2]
                    if rmse < best_score:
#                    if rmse > best_score:

                        best_score, best_cfg = rmse, order
                        r2=r2_score(best_test,best_prediction)
                        print('ARIMA%s RMSE=%.3f R2=%.3f' % (order,rmse,r2))
    
                        
                except:
                    continue
    print('Best ARIMA%s RMSE=%.3f R2=%.3f' % (best_cfg, best_score,r2))
    return best_test,best_prediction


In [17]:
p_values = range(0, 12)
d_values = range(0, 3)
q_values = range(0, 12)

In [18]:

series = y1
warnings.filterwarnings("ignore")
a=evaluate_models(series.values, p_values, d_values, q_values)

ARIMA(0, 0, 1) RMSE=7350.790 R2=-4.157
ARIMA(0, 0, 2) RMSE=5743.919 R2=-2.149
ARIMA(0, 0, 3) RMSE=4554.336 R2=-0.980
ARIMA(0, 0, 4) RMSE=4461.230 R2=-0.899
ARIMA(0, 0, 5) RMSE=3969.557 R2=-0.504
ARIMA(0, 0, 7) RMSE=3915.106 R2=-0.463
ARIMA(0, 0, 8) RMSE=3748.388 R2=-0.341
ARIMA(0, 0, 9) RMSE=3627.428 R2=-0.256
ARIMA(0, 1, 1) RMSE=2954.531 R2=0.167
ARIMA(0, 1, 3) RMSE=2901.648 R2=0.196
ARIMA(0, 1, 4) RMSE=2869.085 R2=0.214
ARIMA(3, 1, 1) RMSE=2859.796 R2=0.219
Best ARIMA(3, 1, 1) RMSE=2859.796 R2=0.219


In [ ]:
series = y1hat
warnings.filterwarnings("ignore")
evaluate_models(series.values, p_values, d_values, q_values)

In [19]:
series = y2
warnings.filterwarnings("ignore")
a=evaluate_models(series.values, p_values, d_values, q_values)

ARIMA(0, 0, 1) RMSE=988.913 R2=-7.029
ARIMA(0, 0, 2) RMSE=747.663 R2=-3.589
ARIMA(0, 0, 3) RMSE=631.501 R2=-2.274
ARIMA(0, 0, 4) RMSE=535.054 R2=-1.350
ARIMA(0, 0, 6) RMSE=516.062 R2=-1.187
ARIMA(0, 0, 7) RMSE=492.772 R2=-0.994
ARIMA(0, 0, 9) RMSE=396.859 R2=-0.293
ARIMA(0, 0, 10) RMSE=393.848 R2=-0.274
ARIMA(0, 1, 1) RMSE=331.237 R2=0.099
ARIMA(0, 1, 2) RMSE=324.892 R2=0.133
ARIMA(0, 1, 3) RMSE=324.301 R2=0.137
ARIMA(1, 1, 1) RMSE=324.075 R2=0.138
ARIMA(10, 1, 0) RMSE=321.941 R2=0.149
Best ARIMA(10, 1, 0) RMSE=321.941 R2=0.149


(array([ 547., 1654., 1326.,  790., 1025., 1175., 1678., 1788., 1773.,
        1469., 1525., 1486., 1245., 1880., 1473., 1355.,  862., 1499.,
        1875., 1474., 1418., 1358., 1521., 1603., 2182., 1446., 1097.,
         873., 1000., 1272., 1899., 1797., 1592., 2150., 1948., 2162.,
        1943., 1818., 1815., 1573., 1608., 2291., 1791., 1998., 1506.,
        1718., 1841., 1977., 1675., 1519., 1234., 1330., 1494., 1373.,
        1726., 1931., 1699., 2109., 1860., 1830., 1877., 1762., 1641.,
        1216., 1808., 1575., 1665.], dtype=float32),
 [array([1425.15865662]),
  array([1248.87811802]),
  array([1165.01674278]),
  array([1221.74218274]),
  array([1226.1788561]),
  array([1014.41625717]),
  array([1065.13638383]),
  array([1415.69678551]),
  array([1915.53820693]),
  array([1649.01941134]),
  array([2109.67740548]),
  array([1238.91026]),
  array([1528.90332965]),
  array([1742.85977184]),
  array([1260.81975403]),
  array([1562.73826332]),
  array([1619.63306362]),
  array([133

In [20]:
series = y3
warnings.filterwarnings("ignore")
a=evaluate_models(series.values, p_values, d_values, q_values)

ARIMA(0, 0, 1) RMSE=1740.188 R2=-2.834
ARIMA(0, 0, 2) RMSE=1375.309 R2=-1.395
ARIMA(0, 0, 3) RMSE=1147.536 R2=-0.667
ARIMA(0, 0, 5) RMSE=951.857 R2=-0.147
ARIMA(0, 0, 6) RMSE=916.658 R2=-0.064
ARIMA(0, 1, 1) RMSE=650.014 R2=0.465
ARIMA(0, 1, 3) RMSE=649.535 R2=0.466
ARIMA(0, 1, 4) RMSE=645.067 R2=0.473
Best ARIMA(0, 1, 4) RMSE=645.067 R2=0.473
